# 4장 기본 신경망 구현

## 4.1 인공신경망의 작동 원리

** 인공신경망** 의 개념은 뇌를 구성하는 신경 세포, 즉 **뉴런**의 동작 원리에 기초한다.

뉴런의 기본 동작은 가지돌기에서 신호를 받아들이고, 이 신호가 축삭돌기를 지나 축삭단말로 전달되는 것이다. 다만 축삭돌기를 지나는 동안 신호가 약해지거나, 너무 약해서 축삭단말까지 전달되지 않거나, 또는 강하게 전달되기도 한다. 

그리고 축삭단말까지 전달된 신호는 연결도니 다음 뉴런의 가지돌기로 전달된다. 우리는 이러한 원리를 지닌 수억 개의 뉴런 조합을 통해 손가락을 움직이건